## Re-run Evaluation

In [50]:
import pandas as pd

ctgan_synthetic_data = []
import os
for dirname, _, filenames in os.walk("..\\thesisgan\\output\\itgan_\\"):
    for filename in filenames:
        if 'syn' in filename:
            fx = os.path.join(dirname, filename)
            ctgan_synthetic_data.append(fx) 
            print(fx)

..\thesisgan\output\ctabgan_sweep_run_1\0\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\1\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\2\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\3\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\4\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\5\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\6\syn.csv
..\thesisgan\output\ctabgan_sweep_run_1\7\syn.csv


In [7]:
real_data = pd.read_csv("..\\thesisgan\\input\\new_hpo_data.csv")
test_data = pd.read_csv("..\\thesisgan\\input\\new_test_data.csv")
import sys
sys.path.append("..")
from ctabganplus.model.evaluation import get_utility_metrics

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
le_dict = {"attack_type": "le_attack_type", "label": "le_label", "proto": "le_proto", "tos": "le_tos"}
for c in le_dict.keys():
    le_dict[c] = LabelEncoder()
    test_data[c] = le_dict[c].fit_transform(test_data[c])
    real_data[c] = le_dict[c].fit_transform(real_data[c])

In [10]:
for synthetic_data in ctgan_synthetic_data:
    df = pd.read_csv(synthetic_data)
    for c in le_dict.keys():
        df[c] = le_dict[c].fit_transform(df[c])
    #if the synthetic data has only one unique value for the attack type, add a row with the attack type
    sample_data_value_counts = df["attack_type"].value_counts()
    for i in range(len(sample_data_value_counts)):
        if sample_data_value_counts[i] == 1:
            at = sample_data_value_counts.index[i]
            df = pd.concat([df,real_data[real_data["attack_type"] == at].sample(3)], ignore_index=True)
    for at in test_data["attack_type"].unique():
        if at not in df["attack_type"].unique():
        #add a row with the attack type
            df = pd.concat([df,real_data[real_data["attack_type"] == at].sample(3)], ignore_index=True)

    result_df, cr = get_utility_metrics(real_data, test_data, df, scaler="MinMax",type={"Classification":["xgb","lr","dt","rf","mlp"]})
    result_df["id"] = synthetic_data.split("\\")[4]
    cr["id"] = synthetic_data.split("\\")[4]
    path = synthetic_data.split("syn")[0]
    print(path)
    result_df.to_csv(path + "new_classification_results.csv", index=False)
    cr.to_csv(path + "new_classification_report.csv", index=False)
    print("Done for " + synthetic_data)

Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data
Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data
Model:  rf trained on fake data
Model:  mlp trained on fake data
..\thesisgan\output\ctabgan_sweep_run_1\0\
Done for ..\thesisgan\output\ctabgan_sweep_run_1\0\syn.csv
Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained on real data
Model:  xgb trained on fake data
Model:  lr trained on fake data
Model:  dt trained on fake data
Model:  rf trained on fake data
Model:  mlp trained on fake data
..\thesisgan\output\ctabgan_sweep_run_1\1\
Done for ..\thesisgan\output\ctabgan_sweep_run_1\1\syn.csv
Model:  xgb trained on real data
Model:  lr trained on real data
Model:  dt trained on real data
Model:  rf trained on real data
Model:  mlp trained

# HPO Results Evaluation

In [71]:
import pandas as pd

itgan_classification_results = []
itgan_classification_report = []
import os
for dirname, _, filenames in os.walk("/work/sangtani/thesis-project/thesisgan/output/ITGAN_HPO/"):
    for filename in filenames:
        if "Classification Report" in filename:
            fx = os.path.join(dirname, filename)
            itgan_classification_report.append(fx)            
            print(fx)
        elif "Classification Results" in filename:
            fx = os.path.join(dirname, filename)
            itgan_classification_results.append(fx)
            print(fx)

/work/sangtani/thesis-project/thesisgan/output/ITGAN_HPO/6/table/Classification Report_55_9b094757f60eee8b5512.table.json
/work/sangtani/thesis-project/thesisgan/output/ITGAN_HPO/6/table/Classification Results_55_09d80a64f26fb7d1a8c7.table.json
/work/sangtani/thesis-project/thesisgan/output/ITGAN_HPO/3/table/Classification Report_55_da636af6e28a7864241a.table.json
/work/sangtani/thesis-project/thesisgan/output/ITGAN_HPO/3/table/Classification Results_55_af1eec6eef5c52c48182.table.json
/work/sangtani/thesis-project/thesisgan/output/ITGAN_HPO/0/table/Classification Report_55_7bc67196cbac8eda3c72.table.json
/work/sangtani/thesis-project/thesisgan/output/ITGAN_HPO/0/table/Classification Results_55_ed225f523ff2d1463c3f.table.json
/work/sangtani/thesis-project/thesisgan/output/ITGAN_HPO/8/table/Classification Report_55_1d9ade8fedf48a065890.table.json
/work/sangtani/thesis-project/thesisgan/output/ITGAN_HPO/8/table/Classification Results_55_c514ff24632671be501b.table.json
/work/sangtani/thesi

In [72]:
import json
# read every json file into a dataframe and append to a main dataframe
itgan_classification_results_df = pd.DataFrame()
for file in itgan_classification_results:
    cr_df = json.load(open(file))
    cr_df = pd.DataFrame(cr_df["data"], columns=cr_df["columns"])
    itgan_classification_results_df = pd.concat([itgan_classification_results_df, cr_df])
    itgan_classification_results_df.reset_index(drop=True, inplace=True)

In [73]:
pd.set_option('display.max_columns', None)

### Classification Results

In [74]:
itgan_classification_results_df

,Acc,AUC,F1_Score,SE_Acc,SE_AUC,SE_F1,Model,Type,run
0,71.271380,0.942061,0.687912,0.079095,0.000408,0.000810,xgb,Real,mbe8qki3
1,15.145030,0.628322,0.000220,0.062662,0.000845,0.000026,lr,Real,mbe8qki3
2,71.279469,0.820541,0.688037,0.079088,0.000671,0.000810,dt,Real,mbe8qki3
3,71.364228,0.949569,0.688578,0.079018,0.000383,0.000809,rf,Real,mbe8qki3
4,59.987295,0.869854,0.590788,0.085637,0.000588,0.000859,mlp,Real,mbe8qki3
...,...,...,...,...,...,...,...,...,...
145,15.762159,0.111705,0.290313,-0.007771,-0.000248,-0.000046,xgb,Difference,t2cdyvnm
146,-49.286895,-0.175395,-0.474742,-0.021016,0.000150,-0.000847,lr,Difference,t2cdyvnm
147,37.139865,0.232168,0.422886,-0.003797,-0.000189,0.000038,dt,Difference,t2cdyvnm
148,16.668857,0.098755,0.291447,-0.007994,-0.000240,-0.000046,rf,Difference,t2cdyvnm


In [75]:
itgan_classification_results_df.drop(["Model"],axis=1).groupby(["Type","run"]).mean().sort_values(by="F1_Score", ascending=False).head(100)


Acc       AUC  F1_Score    SE_Acc    SE_AUC  \
Type       run                                                           
Real       mbe8qki3  57.809480  0.842069  0.531107  0.077100  0.000579   
           t2cdyvnm  57.809480  0.842069  0.531107  0.077100  0.000579   
           tr442wfj  57.809480  0.842069  0.531107  0.077100  0.000579   
           uvv7umuv  57.809480  0.842069  0.531107  0.077100  0.000579   
           yl7kuvm3  57.809480  0.842069  0.531107  0.077100  0.000579   
           yahi2rlb  57.809480  0.842069  0.531107  0.077100  0.000579   
           hh9i0vk5  57.809480  0.842069  0.531107  0.077100  0.000579   
           ip43snr5  57.809480  0.842069  0.531107  0.077100  0.000579   
           5bq07cyo  57.809480  0.842069  0.531107  0.077100  0.000579   
           1m7msbiu  57.809480  0.842069  0.531107  0.077100  0.000579   
Fake       uvv7umuv  53.986365  0.786713  0.461471  0.084850  0.000685   
           1m7msbiu  45.122141  0.705059  0.449267  0.084303  0.000773   
           5bq07cyo  56.960641  0.799204  0.421945  0.086241  0.000688   
           hh9i0vk5  51.140814  0.777747  0.416324  0.084381  0.000688   
           tr442wfj  46.989356  0.743589  0.406952  0.086235  0.000747   
           ip43snr5  54.925071  0.831133  0.405803  0.085462  0.000617   
           yl7kuvm3  40.493071  0.782989  0.395308  0.084271  0.000693   
           t2cdyvnm  53.282806  0.790507  0.393097  0.085363  0.000681   
           yahi2rlb  42.095409  0.666281  0.353248  0.081996  0.000805   
           mbe8qki3  35.945076  0.764365  0.334381  0.080080  0.000712   
Difference mbe8qki3  21.864405  0.077704  0.196726 -0.002979 -0.000133   
           yahi2rlb  15.714072  0.175788  0.177859 -0.004896 -0.000226   
           t2cdyvnm   4.526674  0.051563  0.138010 -0.008263 -0.000102   
           yl7kuvm3  17.316409  0.059080  0.135799 -0.007171 -0.000114   
           ip43snr5   2.884410  0.010936  0.125304 -0.008362 -0.000038   
           tr442wfj  10.820124  0.098480  0.124155 -0.009135 -0.000168   
           hh9i0vk5   6.668666  0.064322  0.114783 -0.007281 -0.000110   
           5bq07cyo   0.848840  0.042865  0.109162 -0.009141 -0.000109   
           1m7msbiu  12.687340  0.137011  0.081840 -0.007203 -0.000194   
           uvv7umuv   3.823115  0.055356  0.069636 -0.007750 -0.000106   

                        SE_F1  
Type       run                 
Real       mbe8qki3  0.000663  
           t2cdyvnm  0.000663  
           tr442wfj  0.000663  
           uvv7umuv  0.000663  
           yl7kuvm3  0.000663  
           yahi2rlb  0.000663  
           hh9i0vk5  0.000663  
           ip43snr5  0.000663  
           5bq07cyo  0.000663  
           1m7msbiu  0.000663  
Fake       uvv7umuv  0.000848  
           1m7msbiu  0.000846  
           5bq07cyo  0.000860  
           hh9i0vk5  0.000844  
           tr442wfj  0.000851  
           ip43snr5  0.000856  
           yl7kuvm3  0.000849  
           t2cdyvnm  0.000844  
           yahi2rlb  0.000810  
           mbe8qki3  0.000805  
Difference mbe8qki3 -0.000142  
           yahi2rlb -0.000147  
           t2cdyvnm -0.000181  
           yl7kuvm3 -0.000186  
           ip43snr5 -0.000193  
           tr442wfj -0.000188  
           hh9i0vk5 -0.000181  
           5bq07cyo -0.000197  
           1m7msbiu -0.000183  
           uvv7umuv -0.000185

# HPO Results Evaluation

In [ ]:
import pandas as pd

ctgan_classification_report = []
ctgan_classification_results = []
import os
for dirname, _, filenames in os.walk("..\\thesisgan\\output\\CTGAN_HPO\\"):
    for filename in filenames:
        if "new_classification_report" in filename:
            fx = os.path.join(dirname, filename)
            ctgan_classification_report.append(fx)            
            print(fx)
        elif "new_classification_results" in filename:
            fx = os.path.join(dirname, filename)
            ctgan_classification_results.append(fx)
            print(fx)

..\thesisgan\output\CTGAN_HPO\0\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\0\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\1\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\1\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\10\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\10\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\11\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\11\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\12\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\12\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\13\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\13\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\14\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\14\new_classification_results.csv
..\thesisgan\output\CTGAN_HPO\15\new_classification_report.csv
..\thesisgan\output\CTGAN_HPO\15\new_classification_

In [ ]:
import json

In [ ]:
# read every json file into a dataframe and append to a main dataframe
# before appending, add the id j0cor82r from the path of the file offline-run-20240509_081254-j0cor82r for each

ctgan_classification_report_df = pd.DataFrame()
for file in ctgan_classification_report:
    cr_df = pd.read_csv(file)
    cr_df["id"] = file.split("\\")[4]
    ctgan_classification_report_df = pd.concat([ctgan_classification_report_df, cr_df])
    ctgan_classification_report_df.reset_index(drop=True, inplace=True)

In [ ]:
pd.set_option("display.max_rows", None)

### Classification Results

In [ ]:
ctgan_classification_results_df = pd.DataFrame()
for file in ctgan_classification_results:
    cr_df = pd.read_csv(file)
    cr_df["id"] = file.split("\\")[4]
    ctgan_classification_results_df = pd.concat([ctgan_classification_results_df, cr_df])
    ctgan_classification_results_df.reset_index(drop=True, inplace=True)

In [ ]:
ctgan_classification_results_df[ctgan_classification_results_df["Type"] == "Fake"].sort_values(by="F1_Score", ascending=False).sort_values(by="id", ascending=False).head(100)

,Acc,AUC,F1_Score,Model,Type,id
234,73.168639,0.888510,0.676596,rf,Fake,9
233,57.923670,0.737023,0.389336,dt,Fake,9
232,64.025036,0.836684,0.589617,lr,Fake,9
235,71.790143,0.924916,0.686023,mlp,Fake,9
221,33.358948,0.583493,0.278108,dt,Fake,8
222,49.351912,0.838198,0.505383,rf,Fake,8
223,49.215455,0.857012,0.523632,mlp,Fake,8
220,40.383374,0.760958,0.405967,lr,Fake,8
209,35.258814,0.595368,0.301022,dt,Fake,7
208,50.006385,0.697204,0.525267,lr,Fake,7


In [ ]:
ctgan_classification_results_df.drop(["Model"],axis=1).groupby(["id", "Type"]).mean().sort_values(by="F1_Score", ascending=False).head(100)


,,Acc,AUC,F1_Score
id,Type,,,
9,Real,62.932143,0.855187,0.629673
14,Real,62.932143,0.855187,0.629673
15,Real,62.932143,0.855187,0.629673
13,Real,62.932143,0.855187,0.629673
6,Real,62.932143,0.855187,0.629673
16,Real,62.932143,0.855187,0.629673
12,Real,62.932143,0.855187,0.629673
17,Real,62.932143,0.855187,0.629673
8,Real,62.932143,0.855187,0.629673
